In [1]:
!python3 --version

Python 3.8.10


In [2]:
import os
import itertools

import pandas
import numpy
import pycytominer

from utils import calculate_percent_replicating_MOA

In [3]:
def do_feature_select(plate_df):
    feature_select_features = pycytominer.cyto_utils.infer_cp_features(
        plate_df
    )

    return pycytominer.feature_select(
        profiles=plate_df,
        features=feature_select_features,
        operation=['variance_threshold','correlation_threshold',
        'drop_na_columns','blocklist']
    )

In [4]:
def all_combo_dropouts_in_a_category(category_list,outfile,subsample=None):
    to_try = [y for x in range(len(category_list)) for y in list(set(itertools.combinations(category_list,x)))]

    if os.path.exists(outfile):
        dropout_df = pandas.read_csv(outfile)
        tried=list(dropout_df['Dropout'])
        print(f"Already try {tried},skipping")
        to_try = [x for x in to_try if str(x) not in tried]
    else:
        dropout_df = pandas.DataFrame()
    for eachdropout in to_try:
        normalized_master = pandas.read_csv('../profiles-pilots/profiles/Stain5_CondC_Standard/BR00120274/BR00120274_normalized_negcon.csv.gz')
        col_list = normalized_master.columns
        if subsample:
             for each_item in subsample:
                metadata_list = [x for x in col_list if 'Metadata' in x] 
                col_list = metadata_list+[x for x in col_list if each_item in x]
        for each_item in eachdropout:
            col_list = [x for x in col_list if each_item not in x]
        dropout_dict = {}
        temp_dropout_dict = {}
        for eachplate in range(70,78):
            try:
                normalized = pandas.read_csv(
                    f'../profiles-pilots/profiles/Stain5_CondC_Standard/BR001202{eachplate}/BR001202{eachplate}_normalized_negcon.csv.gz',
                    usecols=col_list)
                feature_select = do_feature_select(normalized)
                temp_dropout_dict[f'{eachplate}_featnum']=len(feature_select.columns)
                dropout_dict[f'BR001202{eachplate}']= calculate_percent_replicating_MOA('','',data_df=feature_select)
            except:
                dropout_dict[f'BR001202{eachplate}']=0
        results_array = numpy.array(list(dropout_dict.values()))
        for k,v in temp_dropout_dict.items():
            dropout_dict[k]=v
        dropout_dict['Mean']=results_array.mean()
        dropout_dict['Median']=numpy.median(results_array)
        dropout_dict['Standard deviation']=numpy.std(results_array)
        dropout_dict['Dropout']=eachdropout
        dropout_dict['Remaining'] = [x for x in category_list if x not in eachdropout]
        dropout_dict['n_columns']=len(col_list)
        dropout_df = dropout_df.append(dropout_dict,ignore_index=True)
        print(eachdropout,dropout_dict['Mean'])
        dropout_df.to_csv(outfile,index=False)
    print(dropout_df['Mean'].describe())
    print(dropout_df['Median'].describe())

In [5]:
all_combo_dropouts_in_a_category(['Nuclei','Cells','Cytoplasm'],
                                 'checkpoint_csvs/compartment_dropout_combinations.csv')

Already try ['()', "('Cytoplasm',)", "('Cells',)", "('Nuclei',)", "('Nuclei', 'Cells')", "('Cells', 'Cytoplasm')", "('Nuclei', 'Cytoplasm')"],skipping
count    7.000000
mean     0.525397
std      0.022305
min      0.495833
25%      0.506250
50%      0.536111
75%      0.540972
max      0.551389
Name: Mean, dtype: float64
count    7.000000
mean     0.530159
std      0.019207
min      0.500000
25%      0.516667
50%      0.538889
75%      0.544444
max      0.550000
Name: Median, dtype: float64


In [6]:
all_combo_dropouts_in_a_category(['DNA','RNA','ER','Mito','AGP','Brightfield','BFLow','BFHigh'],
                                 'checkpoint_csvs/channel_dropout_combinations.csv')

Already try ['()', "('Mito',)", "('RNA',)", "('Brightfield',)", "('DNA',)", "('BFLow',)", "('BFHigh',)", "('ER',)", "('AGP',)", "('DNA', 'ER')", "('DNA', 'RNA')", "('RNA', 'ER')", "('ER', 'Mito')", "('ER', 'BFLow')", "('ER', 'Brightfield')", "('ER', 'BFHigh')", "('BFLow', 'BFHigh')", "('Brightfield', 'BFHigh')", "('Brightfield', 'BFLow')", "('DNA', 'Mito')", "('DNA', 'BFLow')", "('DNA', 'Brightfield')", "('DNA', 'BFHigh')", "('Mito', 'Brightfield')", "('Mito', 'BFLow')", "('Mito', 'BFHigh')", "('RNA', 'Brightfield')", "('AGP', 'BFLow')", "('AGP', 'BFHigh')", "('ER', 'AGP')", "('RNA', 'BFLow')", "('RNA', 'AGP')", "('RNA', 'Mito')", "('RNA', 'BFHigh')", "('AGP', 'Brightfield')", "('Mito', 'AGP')", "('DNA', 'AGP')", "('RNA', 'Mito', 'AGP')", "('RNA', 'AGP', 'BFLow')", "('Mito', 'Brightfield', 'BFLow')", "('DNA', 'ER', 'BFLow')", "('DNA', 'Mito', 'Brightfield')", "('DNA', 'ER', 'AGP')", "('ER', 'AGP', 'BFHigh')", "('Mito', 'AGP', 'Brightfield')", "('RNA', 'Mito', 'BFHigh')", "('DNA', 'ER',

In [7]:
all_combo_dropouts_in_a_category(['AreaShape','Correlation','Granularity','Intensity','Neighbors','RadialDistribution','Texture'],
                                 'checkpoint_csvs/feature_dropout_combinations.csv')

Already try ['()', "('Granularity',)", "('Neighbors',)", "('Texture',)", "('RadialDistribution',)", "('AreaShape',)", "('Intensity',)", "('Correlation',)", "('Intensity', 'Neighbors')", "('AreaShape', 'RadialDistribution')", "('Intensity', 'Texture')", "('AreaShape', 'Intensity')", "('Correlation', 'Granularity')", "('AreaShape', 'Neighbors')", "('Correlation', 'RadialDistribution')", "('AreaShape', 'Texture')", "('Correlation', 'Intensity')", "('Neighbors', 'RadialDistribution')", "('Correlation', 'Neighbors')", "('Correlation', 'Texture')", "('Neighbors', 'Texture')", "('Granularity', 'RadialDistribution')", "('AreaShape', 'Correlation')", "('Granularity', 'Intensity')", "('Granularity', 'Neighbors')", "('Granularity', 'Texture')", "('Intensity', 'RadialDistribution')", "('RadialDistribution', 'Texture')", "('AreaShape', 'Granularity')", "('Granularity', 'Neighbors', 'Texture')", "('AreaShape', 'Granularity', 'Intensity')", "('AreaShape', 'Granularity', 'Neighbors')", "('Intensity', 

In [ ]:
for eachchannel in ['DNA','RNA','ER','Mito','AGP','Brightfield','BFLow','BFHigh']:
    print(eachchannel)
    all_combo_dropouts_in_a_category(['AreaShape','Correlation','Granularity','Intensity','Neighbors','RadialDistribution','Texture'],
                                 f'checkpoint_csvs/{eachchannel}_only_feature_dropout_combinations.csv',subsample=[eachchannel])
    

DNA
Already try ['()', "('Neighbors',)", "('Intensity',)", "('Texture',)", "('Correlation',)", "('AreaShape',)", "('RadialDistribution',)", "('Granularity',)", "('Granularity', 'Texture')", "('RadialDistribution', 'Texture')", "('AreaShape', 'RadialDistribution')", "('AreaShape', 'Granularity')", "('Correlation', 'Neighbors')", "('Intensity', 'RadialDistribution')", "('AreaShape', 'Intensity')", "('AreaShape', 'Correlation')", "('Granularity', 'RadialDistribution')", "('AreaShape', 'Neighbors')", "('Granularity', 'Intensity')", "('Correlation', 'Texture')", "('Neighbors', 'Texture')", "('Intensity', 'Neighbors')", "('Granularity', 'Neighbors')", "('AreaShape', 'Texture')", "('Correlation', 'RadialDistribution')", "('Correlation', 'Granularity')", "('Neighbors', 'RadialDistribution')", "('Intensity', 'Texture')", "('Correlation', 'Intensity')", "('AreaShape', 'Granularity', 'RadialDistribution')", "('Granularity', 'Intensity', 'Texture')", "('Correlation', 'Intensity', 'Texture')", "('C

('Neighbors', 'Texture') 0.5208333333333334
('AreaShape', 'Correlation') 0.5222222222222223
('Intensity', 'Neighbors') 0.5263888888888889
('AreaShape', 'Neighbors') 0.5166666666666667
('Neighbors', 'RadialDistribution') 0.4611111111111111
('AreaShape', 'Intensity') 0.5263888888888889
('Correlation', 'Neighbors') 0.5222222222222223
('Granularity', 'RadialDistribution') 0.4569444444444445
('Correlation', 'Intensity') 0.5152777777777777
('AreaShape', 'Granularity') 0.5069444444444444
('Correlation', 'Granularity') 0.5152777777777777
('Intensity', 'Texture') 0.5125
('AreaShape', 'Texture') 0.5263888888888889
('Correlation', 'Granularity', 'Texture') 0.5041666666666667
('Neighbors', 'RadialDistribution', 'Texture') 0.45416666666666666
('Correlation', 'Neighbors', 'Texture') 0.5180555555555555
('AreaShape', 'Granularity', 'Texture') 0.5208333333333334
('Correlation', 'Granularity', 'RadialDistribution') 0.4027777777777778
('Correlation', 'Neighbors', 'RadialDistribution') 0.4236111111111111


() 0.5347222222222222
('Granularity',) 0.5305555555555556
('Correlation',) 0.5347222222222222
('RadialDistribution',) 0.4763888888888889
('Neighbors',) 0.5347222222222222
('AreaShape',) 0.5388888888888889
('Intensity',) 0.5347222222222222
('Texture',) 0.5194444444444445
('Correlation', 'Texture') 0.5277777777777778
('Intensity', 'RadialDistribution') 0.4805555555555555
('AreaShape', 'RadialDistribution') 0.47916666666666674
('Correlation', 'RadialDistribution') 0.4152777777777778
('Granularity', 'Neighbors') 0.525
('Granularity', 'Intensity') 0.5361111111111111
('RadialDistribution', 'Texture') 0.4513888888888889
('Granularity', 'Texture') 0.5222222222222223
('Neighbors', 'Texture') 0.5222222222222223
('AreaShape', 'Correlation') 0.5291666666666667
('Intensity', 'Neighbors') 0.5375
('AreaShape', 'Neighbors') 0.5361111111111111
('Neighbors', 'RadialDistribution') 0.4805555555555556
('AreaShape', 'Intensity') 0.5361111111111111
('Correlation', 'Neighbors') 0.5305555555555556
('Granularit

('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'Neighbors', 'RadialDistribution') 0.3555555555555555
('AreaShape', 'Correlation', 'Intensity', 'Neighbors', 'RadialDistribution', 'Texture') 0.37916666666666665
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'RadialDistribution', 'Texture') 0.0
('AreaShape', 'Correlation', 'Granularity', 'Neighbors', 'RadialDistribution', 'Texture') 0.3486111111111111
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'Neighbors', 'Texture') 0.4930555555555555
count    127.000000
mean       0.464217
std        0.091512
min        0.000000
25%        0.429861
50%        0.491667
75%        0.522222
max        0.538889
Name: Mean, dtype: float64
count    127.000000
mean       0.466098
std        0.090283
min        0.000000
25%        0.427778
50%        0.488889
75%        0.522222
max        0.544444
Name: Median, dtype: float64
Mito
() 0.5402777777777779
('Granularity',) 0.5375
('Correlation',) 0.5361111111111111
('RadialDistrib

/home/ubuntu/.local/lib/python3.8/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


...

('Correlation', 'Granularity', 'Neighbors', 'RadialDistribution', 'Texture') 0.20416666666666666
('Correlation', 'Granularity', 'Intensity', 'Neighbors', 'Texture') 0.4638888888888889
('AreaShape', 'Correlation', 'Granularity', 'Neighbors', 'RadialDistribution') 0.30833333333333335
('AreaShape', 'Intensity', 'Neighbors', 'RadialDistribution', 'Texture') 0.3458333333333333
('Correlation', 'Granularity', 'Intensity', 'Neighbors', 'RadialDistribution') 0.32222222222222224
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'Texture') 0.46249999999999997
('AreaShape', 'Correlation', 'Intensity', 'RadialDistribution', 'Texture') 0.21527777777777776
('AreaShape', 'Correlation', 'Intensity', 'Neighbors', 'Texture') 0.4444444444444444
('AreaShape', 'Correlation', 'Neighbors', 'RadialDistribution', 'Texture') 0.25416666666666665
('AreaShape', 'Granularity', 'Intensity', 'RadialDistribution', 'Texture') 0.39444444444444443
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'RadialDist

/home/ubuntu/.local/lib/python3.8/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/home/ubuntu/.local/lib/python3.8/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


('AreaShape', 'Correlation', 'Granularity', 'RadialDistribution', 'Texture') 0.20416666666666666
('Correlation', 'Intensity', 'Neighbors', 'RadialDistribution', 'Texture') 0.21527777777777776
('Correlation', 'Granularity', 'Intensity', 'RadialDistribution', 'Texture') 0.0
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'Neighbors') 0.46805555555555556
('Correlation', 'Granularity', 'Intensity', 'Neighbors', 'RadialDistribution', 'Texture') 0.0
('AreaShape', 'Granularity', 'Intensity', 'Neighbors', 'RadialDistribution', 'Texture') 0.3958333333333333
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'Neighbors', 'RadialDistribution') 0.3194444444444444
('AreaShape', 'Correlation', 'Intensity', 'Neighbors', 'RadialDistribution', 'Texture') 0.21944444444444444
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'RadialDistribution', 'Texture') 0.0


/home/ubuntu/.local/lib/python3.8/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/home/ubuntu/.local/lib/python3.8/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


('AreaShape', 'Correlation', 'Granularity', 'Neighbors', 'RadialDistribution', 'Texture') 0.20277777777777778
('AreaShape', 'Correlation', 'Granularity', 'Intensity', 'Neighbors', 'Texture') 0.4597222222222222
count    127.000000
mean       0.385794
std        0.105188
min        0.000000
25%        0.341667
50%        0.437500
75%        0.467361
max        0.502778
Name: Mean, dtype: float64
count    127.000000
mean       0.387183
std        0.102596
min        0.000000
25%        0.344444
50%        0.438889
75%        0.466667
max        0.500000
Name: Median, dtype: float64
BFHigh
() 0.5069444444444444
('Granularity',) 0.5013888888888889
('Correlation',) 0.45694444444444443
('RadialDistribution',) 0.3819444444444444
('Neighbors',) 0.5083333333333333
('AreaShape',) 0.5069444444444444
('Intensity',) 0.5041666666666667
('Texture',) 0.5097222222222222
('Correlation', 'Texture') 0.45416666666666666
('Intensity', 'RadialDistribution') 0.38749999999999996
('AreaShape', 'RadialDistributio